# Murcielago System V1 - Training & testing

In [1]:
import torch
import torchaudio
from torch.utils.data import DataLoader
from custom_dataset import GunShotsNoisesDataset, split_dataset
import numpy as np
from cnn import ShotDetectionNetwork, ShotDetectionNetwork2
from torch import nn
from train_test_functions import train_step, test_step, eval_model, accuracy_fn
from torchmetrics import Accuracy

## 1) Data preparing: Dataset and Dataloaders

In [2]:
metadata_file = "./dataset/metadata.xlsx"
audios_dir = "./dataset"
fs = 48000
scales = np.arange(1, 129)
transformation_dict = {"wavelet": "cmor","scales":scales}

GSN_visualize = GunShotsNoisesDataset(metadata_file, audios_dir, transformation_dict, fs, 0.01) #this returns audio waveform to analyse samples

GSN_dataset = GunShotsNoisesDataset(metadata_file, audios_dir, transformation_dict, fs)

In [3]:
batch_size = 4
train_data, test_data = split_dataset(GSN_dataset, 0.04)

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

## 2) Model building

### Primer modelo

In [4]:
torch.manual_seed(42)
mur_cnn_v1 = ShotDetectionNetwork(64)
print(mur_cnn_v1)

ShotDetectionNetwork(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): Conv2d(64, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 1, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): LazyLinear(in_features=0, out_features=1, bias=True)
  )
)


c:\Users\joaqo\anaconda3\envs\Sonido\Lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [5]:
learn_rate = 0.02
loss_fn = nn.BCEWithLogitsLoss()
optim = torch.optim.Adam(mur_cnn_v1.parameters(), lr=learn_rate)
metric = Accuracy(task="binary")

In [6]:
epochs = 50

for epoch in range(epochs):
    print("-----------------------------------------------------------")
    print(f"Epoch: {epoch}")
    train_loss = train_step(mur_cnn_v1, train_dataloader, loss_fn, optim, metric=metric)
    if train_loss < 0.3:
        break
    

-----------------------------------------------------------
Epoch: 0


c:\Users\joaqo\anaconda3\envs\Sonido\Lib\site-packages\pywt\_cwt.py:117: FutureWarning: Wavelets from the family cmor, without parameters specified in the name are deprecated. The name should takethe form cmorB-C where B and C are floats representing the bandwidth frequency and center frequency, respectively (example: cmor1.5-1.0).
  wavelet = DiscreteContinuousWavelet(wavelet)
c:\Users\joaqo\OneDrive\Escritorio\Programacion\Murcielago System\custom_dataset.py:149: UserWarning: ComplexHalf support is experimental and many operators don't support it yet. (Triggered internally at C:\b\abs_fakvb73nko\croot\pytorch-select_1730848725921\work\aten\src\ATen\EmptyTensor.cpp:42.)
  coeffs = torch.tensor(coeffs, dtype=torch.complex32)


Train loss: 0.70475 | Train accuracy: 66.66667175292969%
-----------------------------------------------------------
Epoch: 1
Train loss: 0.46249 | Train accuracy: 81.66666412353516%
-----------------------------------------------------------
Epoch: 2
Train loss: 3.89017 | Train accuracy: 73.54166412353516%
-----------------------------------------------------------
Epoch: 3
Train loss: 0.76645 | Train accuracy: 82.5%
-----------------------------------------------------------
Epoch: 4
Train loss: 0.26013 | Train accuracy: 90.83333587646484%


In [7]:
for p in mur_cnn_v1.parameters():
    print(p.grad)

tensor([[[[ 0.0105,  0.0039,  0.0046,  0.0045],
          [ 0.0097,  0.0034,  0.0037,  0.0049],
          [ 0.0077,  0.0040,  0.0045,  0.0061],
          [ 0.0079,  0.0023,  0.0035,  0.0045]]],


        [[[-0.0229, -0.0279, -0.0281, -0.0270],
          [-0.0253, -0.0306, -0.0304, -0.0305],
          [-0.0266, -0.0339, -0.0343, -0.0350],
          [-0.0277, -0.0334, -0.0347, -0.0350]]],


        [[[-0.0283, -0.0323, -0.0333, -0.0329],
          [-0.0300, -0.0343, -0.0353, -0.0359],
          [-0.0310, -0.0362, -0.0371, -0.0388],
          [-0.0312, -0.0353, -0.0369, -0.0382]]],


        ...,


        [[[-0.0181, -0.0258, -0.0301, -0.0344],
          [-0.0148, -0.0210, -0.0256, -0.0309],
          [-0.0079, -0.0139, -0.0191, -0.0235],
          [-0.0037, -0.0100, -0.0150, -0.0202]]],


        [[[-0.3804, -0.4118, -0.4332, -0.4510],
          [-0.3715, -0.4014, -0.4222, -0.4430],
          [-0.3571, -0.3906, -0.4137, -0.4347],
          [-0.3460, -0.3789, -0.4013, -0.4253]]],


     

### Segundo modelo

In [8]:
torch.manual_seed(42)
mur_cnn_v2 = ShotDetectionNetwork(128)
print(mur_cnn_v2)

ShotDetectionNetwork(
  (conv1): Sequential(
    (0): Conv2d(1, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): Conv2d(128, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 1, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): LazyLinear(in_features=0, out_features=1, bias=True)
  )
)


c:\Users\joaqo\anaconda3\envs\Sonido\Lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [9]:
learn_rate = 0.01
loss_fn = nn.BCEWithLogitsLoss()
optim = torch.optim.Adam(mur_cnn_v2.parameters(), lr=learn_rate)
metric = Accuracy(task="binary")

In [10]:
epochs = 75

for epoch in range(epochs):
    print("-----------------------------------------------------------")
    print(f"Epoch: {epoch}")
    train_loss = train_step(mur_cnn_v2, train_dataloader, loss_fn, optim, metric=metric)
    if train_loss < 0.25:
        break

-----------------------------------------------------------
Epoch: 0
Train loss: 0.75787 | Train accuracy: 53.54166793823242%
-----------------------------------------------------------
Epoch: 1
Train loss: 0.49371 | Train accuracy: 78.75%
-----------------------------------------------------------
Epoch: 2
Train loss: 0.41866 | Train accuracy: 84.79166412353516%
-----------------------------------------------------------
Epoch: 3
Train loss: 3.20891 | Train accuracy: 74.58333587646484%
-----------------------------------------------------------
Epoch: 4
Train loss: 0.69518 | Train accuracy: 87.29166412353516%
-----------------------------------------------------------
Epoch: 5
Train loss: 0.23875 | Train accuracy: 90.625%


In [11]:
for p in mur_cnn_v2.parameters():
    print(p.grad)


tensor([[[[-0.0045, -0.0029, -0.0062, -0.0103],
          [-0.0090, -0.0072, -0.0068, -0.0087],
          [-0.0135, -0.0090, -0.0059, -0.0055],
          [-0.0232, -0.0191, -0.0133, -0.0097]]],


        [[[-0.0124, -0.0113, -0.0156, -0.0208],
          [-0.0156, -0.0147, -0.0161, -0.0195],
          [-0.0175, -0.0141, -0.0134, -0.0171],
          [-0.0271, -0.0245, -0.0213, -0.0211]]],


        [[[-0.0082, -0.0075, -0.0092, -0.0119],
          [-0.0103, -0.0103, -0.0115, -0.0140],
          [-0.0136, -0.0127, -0.0135, -0.0158],
          [-0.0198, -0.0190, -0.0188, -0.0197]]],


        ...,


        [[[-0.0019, -0.0107, -0.0147, -0.0160],
          [ 0.0019, -0.0071, -0.0137, -0.0178],
          [ 0.0069, -0.0039, -0.0131, -0.0202],
          [ 0.0147,  0.0037, -0.0069, -0.0155]]],


        [[[ 0.0084,  0.0093,  0.0108,  0.0129],
          [ 0.0108,  0.0112,  0.0115,  0.0131],
          [ 0.0120,  0.0117,  0.0108,  0.0126],
          [ 0.0177,  0.0174,  0.0154,  0.0154]]],


     

### Tercer modelo

In [12]:
torch.manual_seed(42)
mur_cnn_v3 = ShotDetectionNetwork2(128, 2)
print(mur_cnn_v1)

ShotDetectionNetwork(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): Conv2d(64, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 1, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=16, out_features=1, bias=True)
  )
)


In [13]:
learn_rate = 0.0001
loss_fn = nn.BCEWithLogitsLoss()
optim = torch.optim.Adam(mur_cnn_v3.parameters(), lr=learn_rate)
metric = Accuracy(task="binary")

In [14]:
epochs = 100

for epoch in range(epochs):
    print("-----------------------------------------------------------")
    print(f"Epoch: {epoch}")
    train_loss = train_step(mur_cnn_v3, train_dataloader, loss_fn, optim, metric=metric)
    if train_loss < 0.25:
        break

-----------------------------------------------------------
Epoch: 0
Train loss: 0.62679 | Train accuracy: 58.95833206176758%
-----------------------------------------------------------
Epoch: 1
Train loss: 0.57711 | Train accuracy: 64.79166412353516%
-----------------------------------------------------------
Epoch: 2
Train loss: 0.51708 | Train accuracy: 71.25%
-----------------------------------------------------------
Epoch: 3
Train loss: 0.47164 | Train accuracy: 78.54166412353516%
-----------------------------------------------------------
Epoch: 4
Train loss: 0.43121 | Train accuracy: 80.83333587646484%
-----------------------------------------------------------
Epoch: 5
Train loss: 0.38626 | Train accuracy: 83.33332824707031%
-----------------------------------------------------------
Epoch: 6
Train loss: 0.37297 | Train accuracy: 84.16666412353516%
-----------------------------------------------------------
Epoch: 7
Train loss: 0.32395 | Train accuracy: 85.20833587646484%
----

In [15]:
for p in mur_cnn_v3.parameters():
    print(p.grad)


tensor([[[[-6.0714e-04, -5.3595e-04, -5.2020e-04, -5.0212e-04],
          [-5.7699e-04, -5.0469e-04, -4.9076e-04, -4.7329e-04],
          [-5.4279e-04, -4.7586e-04, -4.6320e-04, -4.4786e-04],
          [-5.1363e-04, -4.4451e-04, -4.3122e-04, -4.1304e-04]]],


        [[[ 1.0991e-03,  9.7429e-04,  9.4748e-04,  9.1442e-04],
          [ 1.0383e-03,  9.0510e-04,  8.8228e-04,  8.5155e-04],
          [ 9.7485e-04,  8.5448e-04,  8.3043e-04,  8.0309e-04],
          [ 9.1645e-04,  7.8767e-04,  7.6434e-04,  7.3165e-04]]],


        [[[-9.1134e-04, -8.1180e-04, -7.9029e-04, -7.6173e-04],
          [-8.5936e-04, -7.5613e-04, -7.3579e-04, -7.0961e-04],
          [-8.0450e-04, -7.1006e-04, -6.9126e-04, -6.6847e-04],
          [-7.5646e-04, -6.5641e-04, -6.3674e-04, -6.0663e-04]]],


        ...,


        [[[ 4.3118e-04,  3.8671e-04,  3.7485e-04,  3.6220e-04],
          [ 4.0481e-04,  3.6336e-04,  3.5329e-04,  3.4159e-04],
          [ 3.8374e-04,  3.4441e-04,  3.3293e-04,  3.2017e-04],
          [ 3

## Resultados modelo 1

In [16]:
metric = Accuracy(task="binary")
test_step(mur_cnn_v1, test_dataloader, loss_fn=loss_fn, metric=metric)

Predicción: tensor([1., 0., 0., 1.]) | Respuesta: tensor([1., 0., 0., 1.])
Predicción: tensor([1., 1., 1., 0.]) | Respuesta: tensor([1., 1., 1., 0.])
Predicción: tensor([0., 0., 0., 1.]) | Respuesta: tensor([0., 0., 0., 1.])
Predicción: tensor([0., 1., 1., 1.]) | Respuesta: tensor([0., 1., 1., 1.])
Predicción: tensor([0., 0., 1., 1.]) | Respuesta: tensor([0., 0., 0., 1.])
Test loss: 0.24759 | Test accuracy: 95.0%


## Resultados modelo 2

In [17]:
metric = Accuracy(task="binary")
test_step(mur_cnn_v2, test_dataloader, loss_fn=loss_fn, metric=metric)

Predicción: tensor([0., 1., 1., 0.]) | Respuesta: tensor([1., 1., 1., 0.])
Predicción: tensor([1., 0., 1., 1.]) | Respuesta: tensor([1., 0., 1., 1.])
Predicción: tensor([0., 0., 1., 1.]) | Respuesta: tensor([0., 0., 0., 1.])
Predicción: tensor([1., 1., 1., 1.]) | Respuesta: tensor([1., 1., 0., 1.])
Predicción: tensor([0., 0., 0., 0.]) | Respuesta: tensor([0., 0., 0., 0.])
Test loss: 0.71695 | Test accuracy: 85.0%


## Resultados modelo 3

In [18]:
metric = Accuracy(task="binary")
test_step(mur_cnn_v3, test_dataloader, loss_fn=loss_fn, metric=metric)

Predicción: tensor([0., 0., 1., 0.]) | Respuesta: tensor([0., 0., 1., 0.])
Predicción: tensor([1., 0., 0., 1.]) | Respuesta: tensor([1., 0., 0., 1.])
Predicción: tensor([0., 1., 0., 0.]) | Respuesta: tensor([0., 1., 0., 0.])
Predicción: tensor([1., 1., 1., 0.]) | Respuesta: tensor([1., 1., 1., 0.])
Predicción: tensor([1., 0., 1., 1.]) | Respuesta: tensor([1., 0., 1., 1.])
Test loss: 0.07072 | Test accuracy: 100.0%


# Guardo Modelos

In [22]:
from pathlib import Path

# 1) Crear el directorio
MODEL_PATH = Path("modelos")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2) Crear al path de guardado
MODEL_NAME = "mur_cnn_v3.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME


# 3) Guardar el model state dict
print(F"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=mur_cnn_v3.state_dict(), f=MODEL_SAVE_PATH)

Saving model to: modelos\mur_cnn_v3.pth
